# New Section

In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
import pandas as pd
import numpy as np
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [70]:
dataset = pd.read_csv('/content/drive/MyDrive/DS/Patient Survival Detection/Dataset.csv')
dataset.head()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,119783,50777,118,0,25.0,31.95,0,Caucasian,F,172.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,79267,46918,118,0,81.0,22.64,1,Caucasian,F,165.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,92056,34377,33,0,19.0,NaN,0,Caucasian,M,188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma


In [71]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91713 entries, 0 to 91712
Columns: 186 entries, encounter_id to apache_2_bodysystem
dtypes: float64(170), int64(8), object(8)
memory usage: 130.1+ MB


In [72]:
dataset.shape

(91713, 186)

In [73]:
dataset.describe(include='all')

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
count,91713.000000,91713.000000,91713.000000,91713.000000,87485.000000,88284.000000,91713.000000,90318,91688,90379.000000,...,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90051,90051
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,10
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Caucasian,M,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cardiovascular,Cardiovascular
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70684,49469,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29999,38816
mean,65606.079280,65537.131464,105.669262,0.086302,62.309516,29.185818,0.183736,NaN,NaN,169.641588,...,0.000857,0.015693,0.225192,0.012989,0.026165,0.007066,0.004132,0.020638,NaN,NaN
std,37795.088538,37811.252183,62.854406,0.280811,16.775119,8.275142,0.387271,NaN,NaN,10.795378,...,0.029265,0.124284,0.417711,0.113229,0.159628,0.083763,0.064148,0.142169,NaN,NaN
min,1.000000,1.000000,2.000000,0.000000,16.000000,14.844926,0.000000,NaN,NaN,137.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
25%,32852.000000,32830.000000,47.000000,0.000000,52.000000,23.641975,0.000000,NaN,NaN,162.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
50%,65665.000000,65413.000000,109.000000,0.000000,65.000000,27.654655,0.000000,NaN,NaN,170.100000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
75%,98342.000000,98298.000000,161.000000,0.000000,75.000000,32.930206,0.000000,NaN,NaN,177.800000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN


In [74]:
dataset.describe(include=['object'])

,ethnicity,gender,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem
count,90318,91688,70304,91601,91713,91713,90051,90051
unique,6,2,15,5,3,8,11,10
top,Caucasian,M,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,Cardiovascular,Cardiovascular
freq,70684,49469,36962,54060,86183,50586,29999,38816


In [75]:
dataset.isna().sum()

encounter_id                      0
patient_id                        0
hospital_id                       0
hospital_death                    0
age                            4228
                               ... 
leukemia                        715
lymphoma                        715
solid_tumor_with_metastasis     715
apache_3j_bodysystem           1662
apache_2_bodysystem            1662
Length: 186, dtype: int64

In [76]:
dataset.fillna(dataset.mean(),inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [77]:
dataset.isna().sum()

encounter_id                      0
patient_id                        0
hospital_id                       0
hospital_death                    0
age                               0
                               ... 
leukemia                          0
lymphoma                          0
solid_tumor_with_metastasis       0
apache_3j_bodysystem           1662
apache_2_bodysystem            1662
Length: 186, dtype: int64

In [78]:
dataset['apache_3j_bodysystem'].value_counts()

Cardiovascular          29999
Neurological            11896
Sepsis                  11740
Respiratory             11609
Gastrointestinal         9026
Metabolic                7650
Trauma                   3842
Genitourinary            2172
Musculoskeletal/Skin     1166
Hematological             638
Gynecological             313
Name: apache_3j_bodysystem, dtype: int64

In [79]:
dataset['apache_3j_bodysystem'].count()

90051

In [80]:
dataset['apache_3j_bodysystem'].fillna('unknown',inplace=True)
dataset['apache_2_bodysystem'].fillna('unknown',inplace=True)
dataset['ethnicity'].fillna('unknown',inplace=True)
dataset['hospital_admit_source'].fillna('unknown',inplace=True)
dataset['icu_admit_source'].fillna('unknown',inplace=True)
dataset['gender'].fillna('unknown',inplace=True)

In [81]:
dataset.isna().sum()

encounter_id                   0
patient_id                     0
hospital_id                    0
hospital_death                 0
age                            0
                              ..
leukemia                       0
lymphoma                       0
solid_tumor_with_metastasis    0
apache_3j_bodysystem           0
apache_2_bodysystem            0
Length: 186, dtype: int64

In [82]:
cat_col_list = dataset.select_dtypes(include=['object']).columns.to_list()
dataset[cat_col_list].isna().sum()

ethnicity                0
gender                   0
hospital_admit_source    0
icu_admit_source         0
icu_stay_type            0
icu_type                 0
apache_3j_bodysystem     0
apache_2_bodysystem      0
dtype: int64

In [83]:
dataset.drop(['encounter_id','hospital_id','patient_id','icu_id','icu_stay_type'], inplace=True, axis=1)

In [84]:
dataset.head()

,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_type,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,0,68.0,22.730000,0,Caucasian,M,180.3,Floor,Floor,CTICU,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,0,77.0,27.420000,0,Caucasian,F,160.0,Floor,Floor,Med-Surg ICU,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,0,25.0,31.950000,0,Caucasian,F,172.7,Emergency Department,Accident & Emergency,Med-Surg ICU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,0,81.0,22.640000,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,CTICU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,0,19.0,29.185818,0,Caucasian,M,188.0,unknown,Accident & Emergency,Med-Surg ICU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma


In [85]:
X = dataset.drop(['hospital_death'], axis=1)
y= dataset['hospital_death']
print(X)
print(y)

             age        bmi  elective_surgery  ethnicity gender  height  \
0      68.000000  22.730000                 0  Caucasian      M   180.3   
1      77.000000  27.420000                 0  Caucasian      F   160.0   
2      25.000000  31.950000                 0  Caucasian      F   172.7   
3      81.000000  22.640000                 1  Caucasian      F   165.1   
4      19.000000  29.185818                 0  Caucasian      M   188.0   
...          ...        ...               ...        ...    ...     ...   
91708  75.000000  23.060250                 0  Caucasian      M   177.8   
91709  56.000000  47.179671                 0  Caucasian      F   183.0   
91710  48.000000  27.236914                 0  Caucasian      M   170.2   
91711  62.309516  23.297481                 0  Caucasian      F   154.9   
91712  82.000000  22.031250                 1  Caucasian      F   160.0   

      hospital_admit_source           icu_admit_source      icu_type  \
0                     Floor

In [86]:
from sklearn.model_selection import train_test_split
X_train_ful, X_test, y_train_ful, y_test = train_test_split(X, y, test_size=0.20, random_state=46, shuffle=True)

In [87]:
X_train, X_val, y_train, y_val = train_test_split(X_train_ful, y_train_ful, test_size=0.15, random_state=46, shuffle=True)

In [88]:
cat_col_list.remove('icu_stay_type')

In [89]:
# prepare input data
def prepare_inputs(X_train, X_val, X_test):
    oe = LabelEncoder()
    for cat_col in cat_col_list:
      oe.fit(X_train[cat_col])
      X_train[cat_col] = oe.transform(X_train[cat_col])
      X_val[cat_col] = oe.transform(X_val[cat_col])
      X_test[cat_col] = oe.transform(X_test[cat_col])
   

In [90]:
# prepare target
def prepare_targets(y_train, y_val, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_val_enc = le.transform(y_val)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_val_enc, y_test_enc

In [91]:
# prepare input data
prepare_inputs(X_train, X_val, X_test)
# prepare output data
y_train_enc,y_val_enc, y_test_enc = prepare_targets(y_train, y_val, y_test)

In [92]:
# # Import label encoder 
# from sklearn import preprocessing
# # label_encoder object knows how to understand word labels. 
# label_encoder = preprocessing.OrdinalEncoder()
# # Encode labels in column 'Country'. 
# X_train['ethnicity']= label_encoder.fit_transform(X_train[['ethnicity']]) 
# X_train['gender']= label_encoder.fit_transform(X_train[['gender']]) 
# X_train['hospital_admit_source']= label_encoder.fit_transform(X_train[['hospital_admit_source']])
# X_train['icu_admit_source']= label_encoder.fit_transform(X_train[['icu_admit_source']]) 
# X_train['icu_type']= label_encoder.fit_transform(X_train[['icu_type']]) 
# X_train['apache_3j_bodysystem']= label_encoder.fit_transform(X_train[['apache_3j_bodysystem']])
# X_train['apache_2_bodysystem']= label_encoder.fit_transform(X_train[['apache_2_bodysystem']])
# X_train.head()

In [93]:
# # Import label encoder 
# from sklearn import preprocessing
# # label_encoder object knows how to understand word labels. 
# label_encoder = preprocessing.OrdinalEncoder()
# # Encode labels in column 'Country'. 
# label_encoder.fit(X_train['ethnicity'])
# label_encoder.fit(X_train['gender'])
# label_encoder.fit(X_train['hospital_admit_source'])
# label_encoder.fit(X_train['icu_admit_source'])
# label_encoder.fit(X_train['icu_type'])
# label_encoder.fit(X_train['apache_3j_bodysystem'])
# label_encoder.fit(X_train['apache_2_bodysystem'])
# label_encoder.classes_
# X_train['ethnicity']= label_encoder.transform(X_train['ethnicity']) 
# X_train['gender']= label_encoder.transform(X_train['gender']) 
# X_train['hospital_admit_source']= label_encoder.transform(X_train['hospital_admit_source'])
# X_train['icu_admit_source']= label_encoder.transform(X_train['icu_admit_source']) 
# X_train['icu_type']= label_encoder.transform(X_train['icu_type']) 
# X_train['apache_3j_bodysystem']= label_encoder.transform(X_train['apache_3j_bodysystem'])
# X_train['apache_2_bodysystem']= label_encoder.transform(X_train['apache_2_bodysystem'])
# X_train.head()

In [94]:
# # X_test['ethnicity']= label_encoder.transform(X_test['ethnicity']) 
# X_test['gender']= label_encoder.transform(X_test['gender']) 
# X_test['hospital_admit_source']= label_encoder.transform(X_test['hospital_admit_source'])
# X_test['icu_admit_source']= label_encoder.transform(X_test['icu_admit_source']) 
# X_test['icu_type']= label_encoder.transform(X_test['icu_type']) 
# X_test['apache_3j_bodysystem']= label_encoder.transform(X_test['apache_3j_bodysystem'])
# X_test['apache_2_bodysystem']= label_encoder.transform(X_test['apache_2_bodysystem'])
# # X_test.head()

In [95]:
# define min max scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# transform data
scaled_train = scaler.fit_transform(X_train)
scaled_val = scaler.transform(X_val)
scaled_test = scaler.transform(X_test)


In [96]:
scaled_train.shape[1]

180

In [97]:
model = keras.Sequential()
model.add(Dense(1024, input_dim=scaled_train.shape[1],activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.summary()
# TODO: Compile the model using a loss function and an optimizer.
# opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 1024)              185344    
                                                                 
 dropout_4 (Dropout)         (None, 1024)              0         
                                                                 
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [98]:
# patience - how many epochs to wait before stop training
# if there is no further improvement 
# monitor_val_acc = EarlyStopping(monitor = 'val_loss', patience = 5)
callback = EarlyStopping(monitor='loss', patience=3)
# Complete the model fit operation

#Also we set up big epochs size, just to test that easly stopping is working
model.fit(scaled_train, y_train_enc, batch_size=300, epochs=30, validation_data=(scaled_val, y_val_enc), callbacks=[callback], verbose=1)

Epoch 1/30
208/208 [==============================] - 10s 46ms/step - loss: 0.0000e+00 - accuracy: 0.9114 - val_loss: 0.0000e+00 - val_accuracy: 0.9145
Epoch 2/30
208/208 [==============================] - 9s 43ms/step - loss: 0.0000e+00 - accuracy: 0.9132 - val_loss: 0.0000e+00 - val_accuracy: 0.9145
Epoch 3/30
208/208 [==============================] - 9s 43ms/step - loss: 0.0000e+00 - accuracy: 0.9132 - val_loss: 0.0000e+00 - val_accuracy: 0.9145
Epoch 4/30
208/208 [==============================] - 9s 43ms/step - loss: 0.0000e+00 - accuracy: 0.9132 - val_loss: 0.0000e+00 - val_accuracy: 0.9145


In [99]:
pred = model.predict(scaled_test)
pred

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [100]:
# evaluate the model
scores = model.evaluate(scaled_test, y_test_enc)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


574/574 [==============================] - 2s 4ms/step - loss: 0.0000e+00 - accuracy: 0.9148
accuracy: 91.48%
